This script is an example of how you could generate features from the product images. It also shows a simple way to read the images using ImageReader class.

Here, we're using the last layer of a convolutional network named Resnet50 as the representatiion of the products.
To run this script you need tensorflow instaled on you env, as well as, the numpy, pandas and PIL packages.

In order to this script to work, we supose that the dataset images folder was unpacked in ../data/images

This code could take some hours to run depending if you are using CPU or GPU.

In [2]:
import datetimeprint(list())
print(datetime.datetime.now())

2022-05-15 11:56:33.319306


In [3]:
import os
import time
from pathlib import Path
from typing import Any, Dict, Set, Tuple, Union, List

import numpy as np
import pandas as pd
from PIL import UnidentifiedImageError
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array

IMAGES_ROOT_PATH = "../data/images"

class ImageReader:
    """Load product images from the uncompressed dir you downloaded the images file."""

    def __init__(self, images_root_path=IMAGES_ROOT_PATH, items_metadata="../data/products.parquet"):
        self._images_root_path = images_root_path
        self.image_size = (224, 224)
        if not Path(images_root_path).exists():
            raise ValueError(f"Path {images_root_path} does not exist")
        self.products_image_path = self._load_products_image_path(images_root_path, items_metadata)
        
    def _load_products_image_path(self, images_root_path: str, items_metadata: str):
        items_metadata = pd.read_parquet(items_metadata)
        return {
            product_id: f"{images_root_path}/{path}"
            for product_id, path in zip(items_metadata["product_id"], items_metadata["product_image_path"])
        }

    def exists(self, product_id: int) -> bool:
        """Verify that the cache has the given product ID.
        Args:
            product_id: int that represents the product id
        Returns: True if the product id is on cache.
        """
        path = self._local_image_path(product_id=product_id)
        return os.path.exists(path)

    def get(self, product_id: int, target_size: Tuple[int, int]) -> bytearray:
        """Get the image corresponding to the given product ID.
        Args:
            product_id: int that represents the product id
            target_size: Tuple[int, int] of the returned image
        Returns: bytearray representing the image.
        """
        path = self._local_image_path(product_id=product_id)
        try:
            img = image.load_img(path=path, target_size=target_size)
            return img
        except UnidentifiedImageError as image_error:
            print(f"error opening {path}")
            raise ValueError(image_error) from image_error
        except FileNotFoundError as image_error:
            print(f"FileNotFoundError {path}")
            raise ValueError(image_error) from image_error
    
    def get_batch(self, product_ids: List[int]):
        images = []
        products_with_images = []
        for product_id in product_ids:
            try:
                image = self.get(
                    product_id=product_id,
                    target_size=self.image_size,
                )
                image = img_to_array(image)
                images.append(image)
                products_with_images.append(product_id)
            except (IOError, ValueError) as ex:
                pass
        
        return np.stack(images), products_with_images

    def _local_image_path(
        self, product_id: Union[int, np.int64]
    ) -> Path:
        return self.products_image_path.get(product_id)
    
    def get_product_ids(self) -> List[int]:
        return list(self.products_image_path.keys())

In [4]:
from tensorflow.keras import Sequential
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import Flatten

def get_cnn_model(image_size):
    resnet50 = Sequential()
    resnet50.add(
        ResNet50(
            input_shape=image_size,
            include_top=False,
            weights="imagenet",
            pooling="avg",
        )
    )
    resnet50.add(Flatten())
    return resnet50

def batch_predict(resnet50: Sequential, cache: ImageReader):
    product_ids = cache.get_product_ids()
    product_ids_with_embeddings = []
    product_embeddings = []
    for i in range(0, len(product_ids), 128):
        batch = product_ids[i: i+128]
        try:
            batch_images, batch_product_ids = cache.get_batch(batch)
            batch_images = preprocess_input(batch_images)
            batch_predictions = resnet50.predict(batch_images)
            product_embeddings.append(batch_predictions)
            product_ids_with_embeddings += batch_product_ids
        except ValueError:
            pass
    return product_ids_with_embeddings, np.vstack(product_embeddings)

In [5]:
pip install fastparquet

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/sw/arch/Debian10/EB_production/2021/software/Python/3.9.5-GCCcore-10.3.0/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [6]:
cache = ImageReader()

In [16]:
print(cache.get_product_ids()[:10])

[17073270, 17674562, 17678603, 17179699, 15907453, 15929596, 16199541, 16267399, 16271822, 16317754]


In [4]:
cache = ImageReader()
resnet50 = get_cnn_model(image_size=(*cache.image_size, 3))

product_ids_with_embeddings, product_embeddings = batch_predict(resnet50, cache)
embeddings = pd.DataFrame(
    product_embeddings,
    columns=[f"embedding_{x}" for x in range(product_embeddings.shape[1])],
)
embeddings["product_id"] = product_ids_with_embeddings

2022-05-04 10:09:17.533676: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-04 10:09:19.007191: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10437 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:b1:00.0, compute capability: 6.1
2022-05-04 10:09:19.013623: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


94781440/94765736 [==============================] - 1s 0us/step


2022-05-04 10:09:23.770219: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-05-04 10:09:26.051403: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8201


In [5]:
embeddings.head()

,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,embedding_9,...,embedding_2039,embedding_2040,embedding_2041,embedding_2042,embedding_2043,embedding_2044,embedding_2045,embedding_2046,embedding_2047,product_id
0,0.433182,1.854443,0.090870,0.148389,0.071914,0.002903,0.257232,0.902542,0.040609,0.000000,...,0.000000,0.501384,0.117638,0.157475,0.614729,2.202477,0.002830,0.691712,0.820622,17073270
1,0.698827,3.723713,0.000000,0.206025,0.000000,0.262401,0.105127,0.183554,0.000000,0.000000,...,0.028635,0.000000,0.100318,0.022010,0.070894,0.319197,0.047758,0.985728,0.000000,17674562
2,0.269876,4.523061,0.242475,0.104104,0.000000,0.009380,0.018123,0.204233,0.000000,0.000000,...,0.275482,0.000000,0.126053,0.131009,0.073485,0.174457,0.052749,0.281184,0.341619,17678603
3,1.229910,2.637686,0.099903,0.062075,0.097474,0.010637,0.004436,1.193659,0.311293,0.000000,...,0.036052,0.019644,0.434100,0.088981,0.000000,0.355796,0.018523,0.671135,1.482252,17179699
4,1.002953,0.064150,0.000000,0.058865,0.970041,0.016875,0.287381,0.370938,0.000000,0.218199,...,0.000000,0.089278,0.092408,0.000000,1.029852,0.474000,0.002782,0.263994,0.391165,15907453


In [6]:
import time

unique_name = int(time.time())
embeddings.to_parquet(f"../data/image_embeddings_{unique_name}.parquet")

OverflowError: value too large to convert to int

Exception ignored in: 'fastparquet.cencoding.write_thrift'
Traceback (most recent call last):
  File "/home/lukawc/.local/lib/python3.9/site-packages/fastparquet/writer.py", line 894, in write_simple
    foot_size = f.write(fmd.to_bytes())
OverflowError: value too large to convert to int


OverflowError: value too large to convert to int

Exception ignored in: 'fastparquet.cencoding.write_thrift'
Traceback (most recent call last):
  File "/home/lukawc/.local/lib/python3.9/site-packages/fastparquet/writer.py", line 894, in write_simple
    foot_size = f.write(fmd.to_bytes())
OverflowError: value too large to convert to int


OverflowError: value too large to convert to int

...

Exception ignored in: 'fastparquet.cencoding.write_thrift'
Traceback (most recent call last):
  File "/home/lukawc/.local/lib/python3.9/site-packages/fastparquet/writer.py", line 894, in write_simple
    foot_size = f.write(fmd.to_bytes())
OverflowError: value too large to convert to int


OverflowError: value too large to convert to int

Exception ignored in: 'fastparquet.cencoding.write_thrift'
Traceback (most recent call last):
  File "/home/lukawc/.local/lib/python3.9/site-packages/fastparquet/writer.py", line 894, in write_simple
    foot_size = f.write(fmd.to_bytes())
OverflowError: value too large to convert to int


OverflowError: value too large to convert to int

Exception ignored in: 'fastparquet.cencoding.write_thrift'
Traceback (most recent call last):
  File "/home/lukawc/.local/lib/python3.9/site-packages/fastparquet/writer.py", line 894, in write_simple
    foot_size = f.write(fmd.to_bytes())
OverflowError: value too large to convert to int


In [7]:
print(datetime.datetime.now())

2022-05-04 11:08:27.641212
